In [6]:
# ImageFolder 
# Scheduler 
# Transfer learning

import torch 
import torch.nn as nn 
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets , models, transforms
import matplotlib.pyplot as plt 
import time 
import os 
import copy 
from tqdm.notebook import trange, tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mean = np.array([0.485,0.456,0.406])
std = np.array([0.229,0.224,0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    'val' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
}

# import data 
data_dir = 'data\hymenoptera_data'
sets = ['train','val']
image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir,x),
                                            data_transforms[x])
                for x in ['train','val']}

dataloaders = {x : torch.utils.data.DataLoader(image_datasets[x], batch_size =4,
                                                shuffle = True, num_workers = 0)
                for x in ['train','val']}

dataset_sizes = {x:len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes
print(class_names) 


['ants', 'bees']


In [7]:
def train_model(model, criterion, optimizer, scheduler, num_ephochs =25) :
    since = time.time()

    best_model_wts  = copy.deepcopy(model.state_dict())
    best_acc =0.0 

    for epoch in trange(num_ephochs,desc= 'Main loop') :
        print(f'Epoch {epoch}/{num_ephochs-1}')
        print('-' *10)

        # Each epoch has a training and validation phase 
        for phase in tqdm(['train','val'],):
            if phase == 'train' :
                model.train()  # set model to training model
            else :
                model.eval() # set model to evaluate model 
            running_loss =0.0 
            running_corrects = 0

            # Iterate over data. 
            for inputs, labels in dataloaders[phase] :
                inputs = inputs.to(device)
                labels = labels.to(device) 

                # forward 
                # track history if only in train 
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds =torch.max(outputs,1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase 
                    if phase == 'train' :
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                # statistics 
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train' :
                scheduler.step()
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss: .4f} Acc: {epoch_acc:.4f}')

            # deep copy the model 
            if phase == 'val' and epoch_acc > best_acc :
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print() 
    
    time_elapsed = time.time() - since 
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [8]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs,2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

# scheduler 

step_lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=7, gamma = 0.1)

model = train_model( model, criterion, optimizer, step_lr_scheduler, num_ephochs= 20)


#####
# model = models.resnet18(pretrained=True)
# for param in model.parameters() :
#     param.requires_grad = False

# num_ftrs = model.fc.in_features

# model.fc = nn.Linear(num_ftrs,2)
# model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr = 0.001)

# # scheduler 

# step_lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=7, gamma = 0.1)

# model = train_model( model, criterion, optimizer, step_lr_scheduler, num_ephochs= 20)

Main loop:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 0/19
----------


  0%|          | 0/2 [00:00<?, ?it/s]

train Loss:  0.6803 Acc: 0.5492
val Loss:  0.4743 Acc: 0.8366

Epoch 1/19
----------


  0%|          | 0/2 [00:00<?, ?it/s]

...

train Loss:  0.3105 Acc: 0.8730
val Loss:  0.2142 Acc: 0.9281

Epoch 18/19
----------


  0%|          | 0/2 [00:00<?, ?it/s]

train Loss:  0.3590 Acc: 0.8566
val Loss:  0.2171 Acc: 0.9346

Epoch 19/19
----------


  0%|          | 0/2 [00:00<?, ?it/s]

train Loss:  0.3803 Acc: 0.8566
val Loss:  0.2195 Acc: 0.9346

Training complete in 1m 14s
Best val Acc: 0.941176
